In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## 前処理

In [2]:
newsgroups = fetch_20newsgroups()
y_true = newsgroups.target

In [3]:
def number_normalizer(tokens):
    """ Map all numeric tokens to a placeholder.

    For many applications, tokens that begin with a number are not directly
    useful, but the fact that such a token exists can be relevant.  By applying
    this form of dimensionality reduction, some methods may perform better.
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)


class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super().build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))

## 各特徴の類似度

In [4]:
# ①各targetのデータを取得
def getTargetData(X_data):
    X_similar = [0] * 20
    for i in range(len(newsgroups.target_names)):
        X_similar[i] = X_data[y_true == i]
    return X_similar

In [5]:
# ②ランダムにデータを取得し、類似度を計算
def calSimilarity(X_similar):
    similar_list = []
    for _ in range(X_similar.shape[0]):
        rand = np.random.randint(X_similar.shape[0], size=2)
        similar_list.append(np.dot(X_similar[rand[0]], X_similar[rand[1]]))
    
    return np.mean(similar_list)

In [6]:
# ③各targetの平均類似度をプリント
def printSimilarity(X_data):
    X_similar = getTargetData(X_data)
    for i in range(len(newsgroups.target_names)):
        print("{0:25s}の平均類似度：{1:f}".format(newsgroups.target_names[i], calSimilarity(X_similar[i])))

### min_df=5の場合

In [7]:
vectorizer_5 = NumberNormalizingVectorizer(stop_words='english', min_df=5)
X_5 = vectorizer_5.fit_transform(newsgroups.data)
X_5 = X_5.toarray()

print(X_5.shape)

(11314, 23427)


In [8]:
printSimilarity(X_5)

alt.atheism              の平均類似度：0.049245
comp.graphics            の平均類似度：0.033629
comp.os.ms-windows.misc  の平均類似度：0.050144
comp.sys.ibm.pc.hardware の平均類似度：0.050678
comp.sys.mac.hardware    の平均類似度：0.043781
comp.windows.x           の平均類似度：0.039397
misc.forsale             の平均類似度：0.054259
rec.autos                の平均類似度：0.041466
rec.motorcycles          の平均類似度：0.042126
rec.sport.baseball       の平均類似度：0.043962
rec.sport.hockey         の平均類似度：0.049318
sci.crypt                の平均類似度：0.058081
sci.electronics          の平均類似度：0.033432
sci.med                  の平均類似度：0.038968
sci.space                の平均類似度：0.049875
soc.religion.christian   の平均類似度：0.053657
talk.politics.guns       の平均類似度：0.050370
talk.politics.mideast    の平均類似度：0.061614
talk.politics.misc       の平均類似度：0.044696
talk.religion.misc       の平均類似度：0.047166


### min_df=500の場合

In [9]:
vectorizer_500 = NumberNormalizingVectorizer(stop_words='english', min_df=500)
X_500 = vectorizer_500.fit_transform(newsgroups.data)
X_500 = X_500.toarray()

print(X_500.shape)

(11314, 259)


In [10]:
printSimilarity(X_500)

alt.atheism              の平均類似度：0.199925
comp.graphics            の平均類似度：0.176647
comp.os.ms-windows.misc  の平均類似度：0.229895
comp.sys.ibm.pc.hardware の平均類似度：0.228273
comp.sys.mac.hardware    の平均類似度：0.216404
comp.windows.x           の平均類似度：0.182682
misc.forsale             の平均類似度：0.298841
rec.autos                の平均類似度：0.238031
rec.motorcycles          の平均類似度：0.253718
rec.sport.baseball       の平均類似度：0.232987
rec.sport.hockey         の平均類似度：0.209235
sci.crypt                の平均類似度：0.203219
sci.electronics          の平均類似度：0.214792
sci.med                  の平均類似度：0.191009
sci.space                の平均類似度：0.238719
soc.religion.christian   の平均類似度：0.222378
talk.politics.guns       の平均類似度：0.216974
talk.politics.mideast    の平均類似度：0.203326
talk.politics.misc       の平均類似度：0.224254
talk.religion.misc       の平均類似度：0.179699


## 結論
* min_df=5の場合、各特徴のデータの類似度は低いです。
* min_df=500の場合、各特徴のデータの類似度は少し高くなりますが、まだ高い類度ではないと思います。